## Predicting Prescriber induced Overdose

### DATA SCIENCE IMMERSIVE FINAL CAPE STONE

This capestone was initaited with an idea that why are so much more physicians presecribing medications that can lead to overdose. I have worked with patients for more than a year in a clinic as well as a hospital and saw significant amount of patients requesting pain relief medications again and again. I have seen people i know been affected with the chronic and life threatening overdose.

The objective of this capestone is to develop models to identify types of prescribers that are a high-risk for opioid related fatalities and predict most influential opioids leading to deaths.

The public health data was extracted from [CDC](https://wonder.cdc.gov/) I downloaded into individual tab separated text files as dowloading the 3.2GB data at once and extracting needs significant processing power besides I noticed some anomalies when extracting data from Wonder in large due to the system or functionality of wonder tool they have to group the needed data. The otherData was extraced from CMS Medicare [Part D Opioid Prescriber Data](	https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Part-D-Prescriber.html) and [cms](https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/OpioidMap.html)

In this notebook, i am going to analyze the datasets i would use for the prediction. In particular, in this note book i will use the Exploratory Data Analysis(eda) in order to understand the statistical structure of the data. Furthermore, in the following notebooks i will build a predictive machine learning model to predict a certain target variable (prescribers) from other attributes that are linked with the features that i will learn from the eda. The datasets are collected from a real world datasets currently available  and they contains a substantial amount of missing values. In this notebook, i will try to set up my datasets and in 2.1 i will cean all the datasets that i will use in the future and try to understand their relationships and in the next notebook. In Notebook 3.1  i will visualize the most important features and associated ones based on the data collected  that will be used for the next work. All the datasets were collected from different sources and validated. Somedatasets are official querried from respective government institutions using private usercode and adjusted to the subject after cleaning.

**Author : Kiros Gebremariam**

Cohorts of the Data Science Immersive, General Assembly @ Washington DC campus

# Data Cleaning,munging and EDA

Loadding Necessary Libraries

In [1]:
import os                                                                    # os library
import numpy as np                                                           # numpy library
import pandas as pd                                                          # pandas library
import matplotlib.pylab as plt                                               # plotting and visualization library
import seaborn as sns                                                        # plotting
import statsmodels.api as sm                                                 # for running regressions
np.random.seed(2018)                                                         # Random seed for replication 



# Configure visual settings:
plt.style.use('fivethirtyeight')
sns.set(font_scale=2)
cmap = sns.diverging_palette(220, 10, as_cmap=True) # one of the many color mappings from 
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10.0, 8.0) 
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
import os                      # just ensuring the files using as working Directory(pwd)
print(os.listdir())

['.DS_Store', '.git', '.gitignore', '.ipynb_checkpoints', '1.1. Capestone Data setup_1.ipynb', '1.2. Capestone Data Cleaning_1.ipynb', '2.1. Capestone Finaleda.ipynb', 'inputs', 'LICENSE', 'README.md']


In [3]:
# Prescriber related datasets disaggregated by state,county,zip and rate 
pres_info_df = pd.read_csv('https://data.cms.gov/api/views/6wg9-kwip/rows.csv?accessType=DOWNLOAD')
Prescriber_state_df =pd.read_csv('./inputs/Prescriber_state.csv')
Prescriber_county_df =pd.read_csv('./inputs/Prescriber_county.csv')
Prescriber_zip_df =pd.read_csv('./inputs/Prescriber_zip.csv')
psr_df= pd.read_csv('./inputs/prescriber_state_rate.csv')
pcr_df = pd.read_csv('./inputs/prescriber_county_rate.csv')
pzr_df= pd.read_csv('./inputs/prescriber_zip_rate.csv')
# Drugs, Mental health and Deaths datasets different datasources
KFFMentalHealthspendState_df =pd.read_csv('./inputs/KFFMentalHealthspendState.csv')
pop_urban_df =pd.read_csv('./inputs/pop-urban-pct-historical_cleaned_up.csv')
rawdataKFFopioiddeath_df = pd.read_csv('./inputs/rawdataKFFopioiddeath.csv', engine ="python")
Wonder_Drug_Death_Data1_df = pd.read_csv('./inputs/Wonder_Drug_Death_Data_1.csv')
Wonder_Drug_Death_Data2_df = pd.read_csv('./inputs/Wonder_Drug_Death_Data_2.csv')
drugs_df = pd.read_csv('./inputs/opioids.csv')
# crime related data both property and nonviolent
violentcrime_df =pd.read_csv('./inputs/violentcrime19992014.csv')
propertycrime_df=pd.read_csv('./inputs/propertycrime19992014.csv')
tcrime_df =pd.read_csv('./inputs/TotalVncrime.csv')

# National Health Expenditure from 2009-2016
NHE2016_df = pd.read_csv('./inputs/NHE2016.csv',engine='python')

# Checking Data 

In [4]:
pres_info_df.head()[:2]  # the data is about the Medicare part D prescriber summary extracted from cms

,NPI,NPPES Provider Last Name,NPPES Provider First Name,NPPES Provider ZIP Code,NPPES Provider State,Specialty Description,Total Claim Count,Opioid Claim Count,Opioid Prescribing Rate,Extended-Release Opioid Claims,Extended-Release Opioid Prescribing Rate
0,1003000126,ENKESHAFI,ARDALAN,21502.0,MD,Internal Medicine,545,23.0,4.22%,NaN,NaN
1,1003000142,KHALIL,RASHID,43623.0,OH,Anesthesiology,1733,1004.0,57.93%,63.0,6.27%


In [5]:
#  extracted from from CMS Medicare Part D Opioid Prescribing Geographic 2013-2016 Excel file and 
# created CSV files for the three taps 1) state 2) county and 3) zip
# Stored presciber-state file in a variable(df2)
Prescriber_state_df.fillna('National')[:2]   # the data is about the prescriber/Physician_Locator

,State_Name,State_Abbreviation,State_FIPS,2013_Part_ D_Prescribers,2013_Part_Opioid_Prescribers,2013_Opioid Claims,2013_Extended_Release_Opioid_Claims,2013_Overall_Claims,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,...,2016_Part D Opioid Prescribers,2016_Opioid Claims,2016_Extended Release Opioid Claims,2016_Overall_Claims,2016_Opioid_Prescribing_Rate,2016_Extended_Release_Opioid_Prescribing_ Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate,2013_2015_Change_in_Opioid_Prescribing_Rate,2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate
0,National,National,National,"1,037,770","491,500","78,045,683","4,773,286","1,342,096,654",5.82,6.12,...,496350,77716608,5381275,1452978228,5.34,6.92,-0.47,0.80,-0.30,0.59
1,Alabama,AL,1,"12,820","7,422","2,260,284","118,523","29,160,952",7.75,5.24,...,"7,004","2,156,504","120,579","28,882,148",7.47,5.59,-0.28,0.35,-0.21,0.31


In [6]:
# here just filling the NaN value due to the columns mismatach as there is no need fo data on 
# state abrevation and state state FIPs
Prescriber_state_df.fillna("National")[:5] 

,State_Name,State_Abbreviation,State_FIPS,2013_Part_ D_Prescribers,2013_Part_Opioid_Prescribers,2013_Opioid Claims,2013_Extended_Release_Opioid_Claims,2013_Overall_Claims,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,...,2016_Part D Opioid Prescribers,2016_Opioid Claims,2016_Extended Release Opioid Claims,2016_Overall_Claims,2016_Opioid_Prescribing_Rate,2016_Extended_Release_Opioid_Prescribing_ Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate,2013_2015_Change_in_Opioid_Prescribing_Rate,2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate
0,National,National,National,"1,037,770","491,500","78,045,683","4,773,286","1,342,096,654",5.82,6.12,...,496350,77716608,5381275,1452978228,5.34,6.92,-0.47,0.80,-0.30,0.59
1,Alabama,AL,1,"12,820","7,422","2,260,284","118,523","29,160,952",7.75,5.24,...,"7,004","2,156,504","120,579","28,882,148",7.47,5.59,-0.28,0.35,-0.21,0.31
2,Alaska,AK,2,"2,275","1,099","86,517","8,602","1,281,057",6.75,9.94,...,"1,043","82,557","9,074","1,225,327",6.74,10.99,-0.01,1.05,-0.09,0.71
3,Arizona,AZ,4,"20,542","10,510","1,545,138","151,086","22,126,421",6.98,9.78,...,"11,026","1,621,223","173,171","24,891,001",6.51,10.68,-0.47,0.90,-0.20,0.48
4,Arkansas,AR,5,"7,909","4,700","1,128,356","57,519","16,759,116",6.73,5.10,...,"4,854","1,155,171","65,819","18,139,502",6.37,5.70,-0.36,0.60,-0.36,0.49


In [7]:
#Selected the opioid prescribe rate columsn for years 2013-2016 by prescriber state(psr)
# this was extracted from df2 so that it will be important to see for features in the next stages
psr_df.head()[:2]
psr_df.fillna("National")[:2]

,State_Name,State_Abbreviation,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,2014_Opioid_Prescribing_Rate,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2016_Opioid_Prescribing_Rate,2016_Extended_Release_Opioid_Prescribing_ Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,National,5.82,6.12,5.74,6.27,5.52,6.71,5.34,6.92,-0.47,0.80
1,Alabama,AL,7.75,5.24,7.86,5.24,7.54,5.55,7.47,5.59,-0.28,0.35


In [8]:
psr_df.fillna("National")
psr_df.head()[:2]

,State_Name,State_Abbreviation,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,2014_Opioid_Prescribing_Rate,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2016_Opioid_Prescribing_Rate,2016_Extended_Release_Opioid_Prescribing_ Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,NaN,5.82,6.12,5.74,6.27,5.52,6.71,5.34,6.92,-0.47,0.80
1,Alabama,AL,7.75,5.24,7.86,5.24,7.54,5.55,7.47,5.59,-0.28,0.35


In [9]:
psr_changeex_df = psr_df[['State_Name','2013_2016_Change in Opioid Prescribing Rate', '2013_2016_Change in Extended Release Opioid Prescribing Rate ']]
psr_changeex_df .head()[:2]

,State_Name,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,-0.47,0.80
1,Alabama,-0.28,0.35


In [10]:
#Selected the opioid prescribe rate columsn for years 2013-2016 by prescriber county
# pcr_df.head()[:2]
psr_df.fillna("National")[:2]

,State_Name,State_Abbreviation,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,2014_Opioid_Prescribing_Rate,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2016_Opioid_Prescribing_Rate,2016_Extended_Release_Opioid_Prescribing_ Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,National,5.82,6.12,5.74,6.27,5.52,6.71,5.34,6.92,-0.47,0.80
1,Alabama,AL,7.75,5.24,7.86,5.24,7.54,5.55,7.47,5.59,-0.28,0.35


In [11]:
pcr_df.fillna("National")[:2]
# pcr_df.head()[:2]

,State_Name,State_Abbreviation,County Name,State_FIPS,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,2014_Opioid_Prescribing_Rate,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2016_Opioid_Prescribing_Rate,2016_Extended_Release_Opioid_Prescribing_ Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,National,National,National,5.82,6.12,5.74,6.27,5.52,6.71,5.34,6.92,-0.47,0.8
1,Alabama,AL,Autauga,1001,9.8,4.78,9.72,5.85,8.07,7.24,11.47,9.39,1.67,4.61


In [12]:
pcr_changeex_df = pcr_df[['State_Name','2013_2016_Change in Opioid Prescribing Rate', '2013_2016_Change in Extended Release Opioid Prescribing Rate ']]
pcr_changeex_df .head()[:2]

,State_Name,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,-0.47,0.80
1,Alabama,1.67,4.61


In [13]:
Prescriber_county_df.head()[:2]

,State_Name,State_Abbreviation,County Name,State_FIPS,2013_Part_ D_Prescribers,2013_Part_Opioid_Prescribers,2013_Opioid Claims,2013_Extended_Release_Opioid_Claims,2013_Overall_Claims,2013_Opioid_Prescribing_Rate,...,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Part D Prescribers,2015_Part D Opioid Prescribers,2015_Opioid Claims,2015_Extended Release Opioid Claims,2015_Overall_Claims,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2013_2015_Change_in_Opioid_Prescribing_Rate,2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate
0,National,NaN,NaN,NaN,"1,037,770","491,500","78,045,683","4,773,286","1,342,096,654",5.82,...,6.27,"1,090,979","496,383","78,372,855","5,257,481","1,418,618,875",5.52,6.71,-0.30,0.59
1,Alabama,AL,Autauga,1001.0,67,44,"20,482",979,"208,936",9.80,...,5.85,68,43,"15,998","1,159","198,347",8.07,7.24,-1.73,2.46


In [14]:
Prescriber_zip_df.head()[:2] # this is the dataset for the prescriber zip which is one of the dataclassifications by cms

,State_Name,State_Abbreviation,State_FIPS,2013_Part_ D_Prescribers,2013_Part_Opioid_Prescribers,2013_Opioid Claims,2013_Extended_Release_Opioid_Claims,2013_Overall_Claims,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,...,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Part D Prescribers,2015_Part D Opioid Prescribers,2015_Opioid Claims,2015_Extended Release Opioid Claims,2015_Overall_Claims,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2013_2015_Change_in_Opioid_Prescribing_Rate,2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate
0,National,NaN,NaN,"1,037,770","491,500","78,045,683","4,773,286","1,342,096,654",5.82,6.12,...,6.27,"1,090,979","496,383","78,372,855","5,257,481","1,418,618,875",5.52,6.71,-0.30,0.59
1,Alabama,AL,35004.0,18,9,"1,340",19,"18,632",7.19,1.42,...,6.30,17,7,"1,340",39,"16,830",7.96,2.91,0.77,1.49


In [15]:
#Selected the opioid prescribe rate columsn for years 2013-2016 by prescriber county_rate
# pcr_df.head()[:2]
pcr_df.fillna("National")[:2]

,State_Name,State_Abbreviation,County Name,State_FIPS,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,2014_Opioid_Prescribing_Rate,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2016_Opioid_Prescribing_Rate,2016_Extended_Release_Opioid_Prescribing_ Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,National,National,National,5.82,6.12,5.74,6.27,5.52,6.71,5.34,6.92,-0.47,0.8
1,Alabama,AL,Autauga,1001,9.8,4.78,9.72,5.85,8.07,7.24,11.47,9.39,1.67,4.61


In [16]:
pcr_changeex_df = pcr_df[['State_Name','2013_2016_Change in Opioid Prescribing Rate', '2013_2016_Change in Extended Release Opioid Prescribing Rate ']]
pcr_changeex_df .head()[:2]

,State_Name,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,-0.47,0.80
1,Alabama,1.67,4.61


In [17]:
pzr_df.fillna("National")[:2]
# pzr_df.head()[:2] here the data is by the prescriber zib which is prescriber zip rate
# flling the NaN values under the stateabbrevation and stateFips with National

,State_Name,State_Abbreviation,State_FIPS,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,2014_Opioid_Prescribing_Rate,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2016_Opioid Prescribing Rate,2016_Extended Release Opioid Prescribing Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,National,National,5.82,6.12,5.74,6.27,5.52,6.71,5.34,6.92,-0.47,0.8
1,Alabama,AL,35004,7.19,1.42,9.71,6.3,7.96,2.91,7.47,5.61,0.28,4.19


In [18]:
pzr_changeex_df = pzr_df[['State_Name','2013_2016_Change in Opioid Prescribing Rate', '2013_2016_Change in Extended Release Opioid Prescribing Rate ']]
pzr_changeex_df .head()[:2]

,State_Name,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,-0.47,0.80
1,Alabama,0.28,4.19


In [19]:
KFFMentalHealthspendState_df.head()[:2]  # 

,Location,FY2004__SMHA Expenditures Per Capita,FY2005__SMHA Expenditures Per Capita,FY2006__SMHA Expenditures Per Capita,FY2007__SMHA Expenditures Per Capita,FY2008__SMHA Expenditures Per Capita,FY2009__SMHA Expenditures Per Capita,FY2010__SMHA Expenditures Per Capita,FY2011__SMHA Expenditures Per Capita,FY2012__SMHA Expenditures Per Capita,FY2013__SMHA Expenditures Per Capita,Footnotes
0,United States,93.04,99.55,103.53,113.27,121.13,122.90,120.56,123.93,124.99,119.62,NaN
1,Alabama,58.78,60.31,64.43,74.03,79.39,77.89,78.19,78.33,76.27,72.64,NaN


In [20]:
pop_urban_df.head()[:2]

,FIPS,Area Name,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010
0,0,United States,39.6,45.6,51.2,56.1,56.5,64.0,69.9,73.6,73.7,75.2,79.0,80.7
1,1,Alabama,11.9,17.3,21.7,28.1,30.2,43.8,54.8,58.6,60.0,60.4,55.4,59.0


In [21]:
Wonder_Drug_Death_Data1_df.head()[:2]  # looking on the first two rows of the Dataframe

,Notes,State,State Code,Year,Year Code,Gender,Gender Code,Deaths,Population,Crude Rate
0,NaN,Alabama,1.0,1999.0,1999.0,Female,F,84.0,2293259.0,3.7
1,NaN,Alabama,1.0,1999.0,1999.0,Male,M,111.0,2136882.0,5.2


In [22]:
Wonder_Drug_Death_Data1_df.head()[:2]

,Notes,State,State Code,Year,Year Code,Gender,Gender Code,Deaths,Population,Crude Rate
0,NaN,Alabama,1.0,1999.0,1999.0,Female,F,84.0,2293259.0,3.7
1,NaN,Alabama,1.0,1999.0,1999.0,Male,M,111.0,2136882.0,5.2


In [23]:
NHE2016_df.head()[:2]   # USD in millions

,Expenditure Amount (Millions),1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Total National Health Expenditures,"27,214","29,138","31,842","34,595","38,394","41,852","46,081","51,565","58,402",...,"2,295,307","2,399,121","2,495,414","2,598,823","2,689,349","2,797,260","2,879,008","3,026,157","3,200,815","3,337,248"
1,Out of pocket,"12,949","13,357","14,255","15,311","16,928","18,209","18,594","18,537","20,500",...,"290,000","295,184","293,723","299,742","309,973","318,330","325,189","330,068","339,315","352,537"


In [24]:
violentcrime_df.head()[:2]

,Unnamed: 0,State,Population_1999,Violent Crime rate_1999,Murder and nonnegligent manslaughter rate_1999,Legacy rape rate /1_1999,Revised rape rate /2_1999,Robbery rate_1999,Aggravated assault rate_1999,Unnamed: 8_1999,Population_2014,Violent Crime rate_2014,Murder and nonnegligent manslaughter rate_2014,Legacy rape rate /1_2014,Revised rape rate /2_2014,Robbery rate_2014,Aggravated assault rate_2014,Unnamed: 8_2014,Violent Crime rate_change,Robbery rate_change
0,0,Alabama,4369862,490.2,7.9,34.6,NaN,121.2,326.5,NaN,4849377,427.4,5.7,29.6,41.3,96.9,283.4,NaN,-62.8,-24.3
1,1,Alaska,619500,630.8,8.4,83.5,NaN,91.4,447.6,NaN,736732,635.8,5.6,75.3,104.7,85.4,440.2,NaN,5.0,-6.0


In [25]:
violentcrime_df.columns

Index(['Unnamed: 0', 'State', 'Population_1999', 'Violent Crime rate_1999',
       'Murder and nonnegligent manslaughter rate_1999',
       'Legacy rape rate /1_1999', 'Revised rape rate /2_1999',
       'Robbery rate_1999', 'Aggravated assault rate_1999', 'Unnamed: 8_1999',
       'Population_2014', 'Violent Crime rate_2014',
       'Murder and nonnegligent manslaughter rate_2014',
       'Legacy rape rate /1_2014', 'Revised rape rate /2_2014',
       'Robbery rate_2014', 'Aggravated assault rate_2014', 'Unnamed: 8_2014',
       'Violent Crime rate_change', 'Robbery rate_change'],
      dtype='object')

In [26]:
violentcrime_df.drop(['Unnamed: 0','Unnamed: 8_1999','Unnamed: 8_2014'], axis =1, inplace =True)

In [27]:
violentcrime_df.head()[:2]

,State,Population_1999,Violent Crime rate_1999,Murder and nonnegligent manslaughter rate_1999,Legacy rape rate /1_1999,Revised rape rate /2_1999,Robbery rate_1999,Aggravated assault rate_1999,Population_2014,Violent Crime rate_2014,Murder and nonnegligent manslaughter rate_2014,Legacy rape rate /1_2014,Revised rape rate /2_2014,Robbery rate_2014,Aggravated assault rate_2014,Violent Crime rate_change,Robbery rate_change
0,Alabama,4369862,490.2,7.9,34.6,NaN,121.2,326.5,4849377,427.4,5.7,29.6,41.3,96.9,283.4,-62.8,-24.3
1,Alaska,619500,630.8,8.4,83.5,NaN,91.4,447.6,736732,635.8,5.6,75.3,104.7,85.4,440.2,5.0,-6.0


In [28]:
propertycrime_df.head()[:2]

,Unnamed: 0,State,Population_1999,Property crime rate_1999,Burglary rate_1999,Larceny-theft rate_1999,Motor vehicle theft rate_1999,Unnamed: 6_1999,Population_2014,Property crime rate_2014,Burglary rate_2014,Larceny-theft rate_2014,Motor vehicle theft rate_2014
0,0,Alabama,4369862,3922.3,884.4,2737.3,300.6,NaN,4849377,3177.6,819.0,2149.5,209.1
1,1,Alaska,619500,3728.7,611.3,2688.3,429.1,NaN,736732,2760.0,427.6,2096.4,236.0


In [29]:
propertycrime_df.columns

Index(['Unnamed: 0', 'State', 'Population_1999', 'Property crime rate_1999',
       'Burglary rate_1999', 'Larceny-theft rate_1999',
       'Motor vehicle theft rate_1999', 'Unnamed: 6_1999', 'Population_2014',
       'Property crime rate_2014', 'Burglary rate_2014',
       'Larceny-theft rate_2014', 'Motor vehicle theft rate_2014'],
      dtype='object')

In [30]:
propertycrime_df.drop(['Unnamed: 0','Unnamed: 6_1999'], axis =1, inplace =True)

In [31]:
propertycrime_df.head()[:2]

,State,Population_1999,Property crime rate_1999,Burglary rate_1999,Larceny-theft rate_1999,Motor vehicle theft rate_1999,Population_2014,Property crime rate_2014,Burglary rate_2014,Larceny-theft rate_2014,Motor vehicle theft rate_2014
0,Alabama,4369862,3922.3,884.4,2737.3,300.6,4849377,3177.6,819.0,2149.5,209.1
1,Alaska,619500,3728.7,611.3,2688.3,429.1,736732,2760.0,427.6,2096.4,236.0


In [32]:
tcrime_df.head()[:2]

,Unnamed: 0,State,Population_1999_x,Violent Crime rate_1999,Murder and nonnegligent manslaughter rate_1999,Legacy rape rate /1_1999,Revised rape rate /2_1999,Robbery rate_1999,Aggravated assault rate_1999,Unnamed: 8_1999,...,Burglary rate_1999,Larceny-theft rate_1999,Motor vehicle theft rate_1999,Unnamed: 6_1999,Population_2014_y,Property crime rate_2014,Burglary rate_2014,Larceny-theft rate_2014,Motor vehicle theft rate_2014,Property crime rate_change
0,0,Alabama,4369862,490.2,7.9,34.6,NaN,121.2,326.5,NaN,...,884.4,2737.3,300.6,NaN,4849377,3177.6,819.0,2149.5,209.1,-744.7
1,1,Alaska,619500,630.8,8.4,83.5,NaN,91.4,447.6,NaN,...,611.3,2688.3,429.1,NaN,736732,2760.0,427.6,2096.4,236.0,-968.7


In [33]:
tcrime_df.drop(['Unnamed: 0'], axis =1, inplace =True)

In [34]:
tcrime_df.head()[:2]

,State,Population_1999_x,Violent Crime rate_1999,Murder and nonnegligent manslaughter rate_1999,Legacy rape rate /1_1999,Revised rape rate /2_1999,Robbery rate_1999,Aggravated assault rate_1999,Unnamed: 8_1999,Population_2014_x,...,Burglary rate_1999,Larceny-theft rate_1999,Motor vehicle theft rate_1999,Unnamed: 6_1999,Population_2014_y,Property crime rate_2014,Burglary rate_2014,Larceny-theft rate_2014,Motor vehicle theft rate_2014,Property crime rate_change
0,Alabama,4369862,490.2,7.9,34.6,NaN,121.2,326.5,NaN,4849377,...,884.4,2737.3,300.6,NaN,4849377,3177.6,819.0,2149.5,209.1,-744.7
1,Alaska,619500,630.8,8.4,83.5,NaN,91.4,447.6,NaN,736732,...,611.3,2688.3,429.1,NaN,736732,2760.0,427.6,2096.4,236.0,-968.7
